In [2]:
import spacy

In [3]:
nlp=spacy.load("output/model-last/")

In [1]:
texto="Vem o próprio. Sem queixas com a medicação (Faz Osimertib 80mg 1cp/dia). Diz estar bem melhor da diarreia. Vai fazer TAC na próxima semana"

In [4]:
doc=nlp(texto)

In [5]:
from spacy import displacy
colors = {"Estado" : "#5F9EA0", "Terapêutica" : "#00FF00","RAM" : "#FFA07A","MCDT" : "#FF00FF", "Dose" : "#696969", "Posologia" : "#E6E6FA"}
options = {"colors" : colors}
displacy.render(doc,style="ent",jupyter="True", options = options)